# VGG16 model comparison

In this notebook, a modern convolution model called 'VGG16' is applied to the already known Chest X-rays data, which present unbalanced classes.

The power of the VGG16 model has already been demonstrated on these data, so the purpose of this book is to compare the following models:

1) VGG16 with no bias and no class weights 

2) VGG16 with no bias and with class weights

3) VGG16 with bias and with no class weights

4) VGG16 with bias and with class weights

The question to ask is: is proper data engineering on data with unbalanced classes really necessary to obtain better accuracy values?

**RESULTS**: VGG16 with bias and with class weights is the best model, with the following performance:

Loss on test set:  2.613898992538452

Accuracy on test set:  0.7932692170143127

# Step 1: Data reading and insight

In [2]:
import glob
import os
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import sklearn
from tensorflow import keras
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.models import Sequential
from keras import initializers
from keras.preprocessing.image import ImageDataGenerator

seed = 12
np.random.seed(seed)

In [3]:
dir_path = "../input/chest-xray-pneumonia/chest_xray/"

In [4]:
''' training path '''
train_p = os.path.join(dir_path, "train")

''' test path '''
test_p =os.path.join(dir_path, "test")

''' val path '''
val_p = os.path.join(dir_path, "val")

In [5]:
''' PNEUMONIA images '''
p_train_images = glob.glob(train_p + "/PNEUMONIA/*.jpeg")

''' NORMAL  images '''
n_train_images = glob.glob(train_p + "/NORMAL/*.jpeg")

# Step 2: EDA

In [6]:
df = pd.DataFrame(np.concatenate([[0]*len(n_train_images) , [1] *  len(p_train_images)]),columns=["class"])

In [7]:
sns.countplot(df['class'],data=df)

**Examine the class label imbalance**

Let's look at the dataset imbalance:

In [8]:
neg, pos = np.bincount(df['class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

The positive class accounts for 74.29% of total

# Step 3: Data Preparation

ImageDataGenerator's goal is to make it simple to input data with labels into the model. It is a highly useful class since it includes various functions for data augmentation. The most notable benefit of this class is that it has no effect on the data stored on disk. This class changes the data as it is passed to the model.

In [53]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(train_p, target_size = (224,224))

vldata = ImageDataGenerator()
valdata = vldata.flow_from_directory(val_p, target_size = (224,224))

tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(test_p, target_size = (224,224))

# Step 4: Modelling

INFO: the VGG Network can be partitioned into two parts: the first consisting mostly of convolutional and pooling layers and the second consisting of fully connected layers. The convolutional layers are grouped in nonlinear transformations that leave the dimensonality unchanged, followed by a resolution-reduction step (See [https://d2l.ai/chapter_convolutional-modern/vgg.html](http://))

1) VGG16 with no bias and no class weights 

In [9]:
# VGG16 Model

model_1 = Sequential()
model_1.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_1.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_1.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_1.add(Flatten())
model_1.add(Dense(units=4096,activation="relu"))
model_1.add(Dense(units=4096,activation="relu"))
model_1.add(Dense(units=2, activation="softmax"))

Here I will be using Adam optimiser to reach to the global minima while training out model. I set lr = 0.0001 with decay rate = 0.00001

In [10]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001, decay=1e-5)

model_1.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [15]:
batch_size = 32
nb_epochs = 20


# Define the number of training steps
nb_train_steps = total//batch_size

ModelCheckpoint assists us in saving the model by monitoring a specific parameter of the model. In this scenario, I'm tracking validation accuracy by giving "val_acc" to ModelCheckpoint. The model will be stored to disk only if the model's validation accuracy in the current epoch is greater than it was in the previous epoch.

EarlyStopping allows us to end the model's training early if there is no rise in the parameter that I have selected to check in EarlyStopping. In this scenario, I'm tracking validation accuracy by giving "val_acc" to EarlyStopping. I've set patience to 5, which indicates that the model will stop training if it doesn't see an increase in validation accuracy after 5 epochs.

I'm using model.fit generator to feed data to the model because I'm also using ImageDataGenerator. I'll provide train and test data to fit generator.

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_1 = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_1 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
history_1 = model_1.fit_generator(generator=traindata,epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                              validation_data= valdata,callbacks=[checkpoint_1,early_1])

In [13]:
import matplotlib.pyplot as plt
plt.plot(history_1.history["accuracy"])
plt.plot(history_1.history['val_accuracy'])
plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.title("Performance for VGG16 with no bias and no class weights")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [14]:
# Evaluation on test dataset
test_loss, test_score = model_1.evaluate(testdata, batch_size=16)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

2) VGG16 with no bias and with class weights

In [20]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [25]:
# VGG16 Model

model_2 = Sequential()
model_2.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_2.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_2.add(Flatten())
model_2.add(Dense(units=4096,activation="relu"))
model_2.add(Dense(units=4096,activation="relu"))
model_2.add(Dense(units=2, activation="softmax"))

In [26]:
model_2.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [27]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_2 = ModelCheckpoint("vgg16_2.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_2 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
history_2 = model_2.fit_generator(generator=traindata,epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                              validation_data= valdata,callbacks=[checkpoint_2,early_2], class_weight = {0: weight_for_0, 1: weight_for_1})

In [28]:
import matplotlib.pyplot as plt
plt.plot(history_2.history["accuracy"])
plt.plot(history_2.history['val_accuracy'])
plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title("Performance for VGG16 with no bias and class weights")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [29]:
# Evaluation on test dataset
test_loss, test_score = model_2.evaluate(testdata, batch_size=16)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

3) VGG16 with bias and no class weights

In [54]:
initial_bias = np.log([pos/neg])
initial_bias

In [55]:
# VGG16 Model

model_3 = Sequential()
model_3.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_3.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_3.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_3.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_3.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_3.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_3.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_3.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_3.add(Flatten())
model_3.add(Dense(units=4096,activation="relu"))
model_3.add(Dense(units=4096,activation="relu"))
model_3.add(Dense(units=2, activation="softmax", bias_initializer=initializers.Constant(initial_bias)))

In [56]:
model_3.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [57]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_3 = ModelCheckpoint("vgg16_3.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_3 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
history_3 = model_3.fit_generator(generator=traindata,epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                              validation_data= valdata,callbacks=[checkpoint_3,early_3])

In [58]:
import matplotlib.pyplot as plt
plt.plot(history_3.history["accuracy"])
plt.plot(history_3.history['val_accuracy'])
plt.plot(history_3.history['loss'])
plt.plot(history_3.history['val_loss'])
plt.title("Performance for VGG16 with bias and no class weights")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [59]:
# Evaluation on test dataset
test_loss, test_score = model_3.evaluate(testdata, batch_size=16)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

4) VGG16 with bias and with class weights

In [60]:
# VGG16 Model

model_4 = Sequential()
model_4.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_4.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model_4.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_4.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_4.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_4.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_4.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_4.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model_4.add(Flatten())
model_4.add(Dense(units=4096,activation="relu"))
model_4.add(Dense(units=4096,activation="relu"))
model_4.add(Dense(units=2, activation="softmax", bias_initializer=initializers.Constant(initial_bias)))

In [61]:
model_4.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [62]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_4 = ModelCheckpoint("vgg16_4.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_4 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
history_4 = model_4.fit_generator(generator=traindata,epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                                 validation_data= valdata,callbacks=[checkpoint_4,early_4], class_weight = {0: weight_for_0, 1: weight_for_1})


In [63]:
import matplotlib.pyplot as plt
plt.plot(history_4.history["accuracy"])
plt.plot(history_4.history['val_accuracy'])
plt.plot(history_4.history['loss'])
plt.plot(history_4.history['val_loss'])
plt.title("Performance for VGG16 with bias and class weights")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [64]:
# Evaluation on test dataset
test_loss, test_score = model_4.evaluate(testdata, batch_size=16)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

FINAL RESULTS

**MODEL_4 IS THE BEST MODEL**